In [1]:
# This script is for DNN socio variables
# Take Dataset3 LA for example

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch

# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
X_socio_test_passed_1 = pd.read_csv("../Data/01 Processed Dataset/Socio_LA.csv") #input
X_sociodemo_test_passed = X_socio_test_passed_1.set_index('GEOID')
y = X_sociodemo_test_passed.pop('obesity_cr')

In [3]:
len(X_sociodemo_test_passed),X_sociodemo_test_passed.columns

(947,
 Index(['% Black', '% Ame Indi and AK Native', '% Asian',
        '% Nati Hawa and Paci Island', '% Hispanic or Latino', '% male',
        '% married', '% age 18-29', '% age 30-39', '% age 40-49', '% age 50-59',
        '% age >=60', '% <highschool', 'median income', '% unemployment',
        '% below poverty line', '% food stamp/SNAP', 'median value units built',
        'median year units built', '% renter-occupied housing units',
        'population density'],
       dtype='object'))

In [4]:
# New code
# sociodemographic predictors

y_dnn_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

for train_index, test_index in ten_fold.split(X_sociodemo_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_sociodemo_test_passed.iloc[train_index], X_sociodemo_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #scaler = StandardScaler()
    
    #scaled_X_train = scaler.fit_transform(X_train)
    #scaled_X_test = scaler.transform(X_test)
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    tf.random.set_seed(42)
    dnn_model = keras.models.Sequential([
        keras.layers.Dense(160,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(16,activation="relu"),
        keras.layers.Dense(176,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128,activation="relu"),
        keras.layers.Dense(160,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1)
    ])
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    dnn_model.compile(optimizer="adam", loss=keras.losses.mean_absolute_error, metrics=[keras.metrics.mean_squared_error])
    dnn_model.fit(x=scaled_X_train, y=y_train, validation_split=0.1, epochs=50, callbacks=[early_stop], verbose=2)
    
    this_y_predict = dnn_model.predict(scaled_X_test).flatten()

    y_dnn_socio_predict = y_dnn_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [ 23  30  39  44  59  63  67  70  72  76  86  88  96 107 120 136 139 165
 168 198 208 209 215 218 247 250 259 260 265 275 280 292 294 298 310 312
 327 331 332 333 363 365 377 388 394 439 449 453 457 464 465 478 481 495
 500 513 518 519 527 541 554 559 569 589 591 599 616 617 643 644 653 658
 673 679 684 695 708 753 762 767 777 778 783 792 804 816 826 850 858 874
 883 908 926 932 942]
Epoch 1/50
24/24 - 1s - loss: 21.7306 - mean_squared_error: 539.5837 - val_loss: 8.8100 - val_mean_squared_error: 129.7315 - 896ms/epoch - 37ms/step
Epoch 2/50
24/24 - 0s - loss: 7.3131 - mean_squared_error: 93.2766 - val_loss: 5.5944 - val_mean_squared_error: 61.3915 - 76ms/epoch - 3ms/step
Epoch 3/50
24/24 - 0s - loss: 4.9010 - mean_squared_error: 43.0090 - val_loss: 4.2418 - val_mean_squared_error: 36.7843 - 62ms/epoch - 3ms/step
Epoch 4/50
24/24 - 0s - loss: 4.1179 - mean_squared_error: 27.6799 - val_loss: 3.7342 - val_mean_squared_error: 29.6805 - 76ms/epoch - 3ms/step
Epoch 5/50
24/24 - 0s - lo

Epoch 13/50
24/24 - 0s - loss: 2.4473 - mean_squared_error: 9.8756 - val_loss: 1.7546 - val_mean_squared_error: 7.0103 - 66ms/epoch - 3ms/step
Epoch 14/50
24/24 - 0s - loss: 2.2375 - mean_squared_error: 7.9607 - val_loss: 2.1403 - val_mean_squared_error: 7.9914 - 72ms/epoch - 3ms/step
Epoch 15/50
24/24 - 0s - loss: 2.1875 - mean_squared_error: 8.0180 - val_loss: 2.1657 - val_mean_squared_error: 8.3755 - 55ms/epoch - 2ms/step
Epoch 16/50
24/24 - 0s - loss: 2.1583 - mean_squared_error: 7.3763 - val_loss: 1.8130 - val_mean_squared_error: 6.6039 - 52ms/epoch - 2ms/step
Epoch 17/50
24/24 - 0s - loss: 2.0920 - mean_squared_error: 7.2408 - val_loss: 1.4438 - val_mean_squared_error: 5.1221 - 51ms/epoch - 2ms/step
Epoch 18/50
24/24 - 0s - loss: 2.0652 - mean_squared_error: 6.8441 - val_loss: 1.8552 - val_mean_squared_error: 5.9068 - 51ms/epoch - 2ms/step
Epoch 19/50
24/24 - 0s - loss: 2.1042 - mean_squared_error: 7.4960 - val_loss: 1.3946 - val_mean_squared_error: 5.0855 - 49ms/epoch - 2ms/step

Epoch 18/50
24/24 - 0s - loss: 2.0129 - mean_squared_error: 6.8686 - val_loss: 1.8214 - val_mean_squared_error: 6.2209 - 50ms/epoch - 2ms/step
Epoch 19/50
24/24 - 0s - loss: 2.1306 - mean_squared_error: 7.3255 - val_loss: 1.4547 - val_mean_squared_error: 5.3109 - 50ms/epoch - 2ms/step
Epoch 20/50
24/24 - 0s - loss: 1.9376 - mean_squared_error: 6.2017 - val_loss: 1.4845 - val_mean_squared_error: 4.9203 - 50ms/epoch - 2ms/step
Epoch 21/50
24/24 - 0s - loss: 2.0214 - mean_squared_error: 6.5534 - val_loss: 1.4910 - val_mean_squared_error: 4.9747 - 51ms/epoch - 2ms/step
Epoch 22/50
24/24 - 0s - loss: 1.9158 - mean_squared_error: 6.1798 - val_loss: 1.3879 - val_mean_squared_error: 4.0866 - 49ms/epoch - 2ms/step
Epoch 23/50
24/24 - 0s - loss: 1.9507 - mean_squared_error: 6.4558 - val_loss: 1.5746 - val_mean_squared_error: 4.5584 - 49ms/epoch - 2ms/step
Epoch 24/50
24/24 - 0s - loss: 2.0395 - mean_squared_error: 6.9055 - val_loss: 1.2697 - val_mean_squared_error: 3.8294 - 51ms/epoch - 2ms/step

Epoch 23/50
24/24 - 0s - loss: 1.9249 - mean_squared_error: 5.9758 - val_loss: 1.2879 - val_mean_squared_error: 3.8698 - 50ms/epoch - 2ms/step
Epoch 24/50
24/24 - 0s - loss: 2.2457 - mean_squared_error: 8.5395 - val_loss: 1.3002 - val_mean_squared_error: 4.0899 - 51ms/epoch - 2ms/step
Epoch 25/50
24/24 - 0s - loss: 1.9392 - mean_squared_error: 6.1127 - val_loss: 1.4161 - val_mean_squared_error: 4.3928 - 51ms/epoch - 2ms/step
Epoch 26/50
24/24 - 0s - loss: 2.0264 - mean_squared_error: 6.5699 - val_loss: 1.6712 - val_mean_squared_error: 5.0451 - 66ms/epoch - 3ms/step
Epoch 27/50
24/24 - 0s - loss: 1.8685 - mean_squared_error: 5.7569 - val_loss: 1.2994 - val_mean_squared_error: 3.9134 - 60ms/epoch - 2ms/step
Epoch 28/50
24/24 - 0s - loss: 1.8756 - mean_squared_error: 5.5620 - val_loss: 1.5927 - val_mean_squared_error: 4.5633 - 54ms/epoch - 2ms/step
Epoch 29/50
24/24 - 0s - loss: 1.8844 - mean_squared_error: 5.9527 - val_loss: 2.0446 - val_mean_squared_error: 6.2801 - 50ms/epoch - 2ms/step

Epoch 45/50
24/24 - 0s - loss: 1.7873 - mean_squared_error: 5.2154 - val_loss: 2.3801 - val_mean_squared_error: 7.6380 - 53ms/epoch - 2ms/step
Epoch 46/50
24/24 - 0s - loss: 1.7405 - mean_squared_error: 4.9240 - val_loss: 1.3483 - val_mean_squared_error: 3.5513 - 56ms/epoch - 2ms/step
Epoch 47/50
24/24 - 0s - loss: 1.7264 - mean_squared_error: 4.8010 - val_loss: 1.2931 - val_mean_squared_error: 3.9632 - 40ms/epoch - 2ms/step
Epoch 48/50
24/24 - 0s - loss: 1.8845 - mean_squared_error: 5.5609 - val_loss: 1.1260 - val_mean_squared_error: 2.8971 - 55ms/epoch - 2ms/step
Epoch 49/50
24/24 - 0s - loss: 1.8069 - mean_squared_error: 5.4180 - val_loss: 1.3302 - val_mean_squared_error: 3.2048 - 50ms/epoch - 2ms/step
Epoch 50/50
24/24 - 0s - loss: 1.7773 - mean_squared_error: 5.2784 - val_loss: 1.1820 - val_mean_squared_error: 2.8686 - 50ms/epoch - 2ms/step
TEST: [ 17  19  38  46  50  57  68  75  93 116 117 124 126 142 149 153 154 175
 184 188 195 203 236 237 245 263 268 271 277 284 287 304 320 34

Epoch 45/50
24/24 - 0s - loss: 1.8016 - mean_squared_error: 5.4926 - val_loss: 1.3263 - val_mean_squared_error: 2.8529 - 50ms/epoch - 2ms/step
Epoch 46/50
24/24 - 0s - loss: 1.8633 - mean_squared_error: 5.6790 - val_loss: 0.9965 - val_mean_squared_error: 1.6408 - 50ms/epoch - 2ms/step
Epoch 47/50
24/24 - 0s - loss: 1.7754 - mean_squared_error: 5.2819 - val_loss: 0.9364 - val_mean_squared_error: 1.5331 - 65ms/epoch - 3ms/step
Epoch 48/50
24/24 - 0s - loss: 1.8130 - mean_squared_error: 5.3124 - val_loss: 1.1055 - val_mean_squared_error: 2.1159 - 59ms/epoch - 2ms/step
Epoch 49/50
24/24 - 0s - loss: 1.7687 - mean_squared_error: 5.1895 - val_loss: 1.3894 - val_mean_squared_error: 2.9058 - 51ms/epoch - 2ms/step
Epoch 50/50
24/24 - 0s - loss: 1.8294 - mean_squared_error: 5.5252 - val_loss: 1.0254 - val_mean_squared_error: 1.7361 - 53ms/epoch - 2ms/step
TEST: [  8  16  26  36  37  45  48  53 103 111 115 119 127 150 151 152 157 162
 171 180 190 194 207 225 226 229 253 255 262 283 297 301 303 32

Epoch 45/50
24/24 - 0s - loss: 1.8018 - mean_squared_error: 5.4158 - val_loss: 2.3355 - val_mean_squared_error: 7.1176 - 64ms/epoch - 3ms/step
Epoch 46/50
24/24 - 0s - loss: 1.7812 - mean_squared_error: 5.1970 - val_loss: 1.2159 - val_mean_squared_error: 2.9929 - 60ms/epoch - 2ms/step
TEST: [112 122 123 129 143 146 147 183 186 197 202 219 224 232 233 246 258 267
 279 282 293 317 325 347 348 358 373 376 384 386 400 402 410 415 438 441
 463 472 488 496 505 509 550 556 607 608 609 619 624 627 632 637 639 641
 645 648 659 671 676 691 696 709 712 720 728 730 743 744 750 794 797 798
 801 807 809 823 836 839 849 855 861 863 865 868 877 879 880 888 901 906
 919 928 934 940]
Epoch 1/50
24/24 - 1s - loss: 21.6022 - mean_squared_error: 542.6696 - val_loss: 9.4205 - val_mean_squared_error: 148.0146 - 864ms/epoch - 36ms/step
Epoch 2/50
24/24 - 0s - loss: 8.2928 - mean_squared_error: 118.8923 - val_loss: 6.4257 - val_mean_squared_error: 85.7829 - 72ms/epoch - 3ms/step
Epoch 3/50
24/24 - 0s - loss: 5

Epoch 15/50
24/24 - 0s - loss: 2.2715 - mean_squared_error: 8.8840 - val_loss: 2.5349 - val_mean_squared_error: 9.5911 - 67ms/epoch - 3ms/step
Epoch 16/50
24/24 - 0s - loss: 2.1080 - mean_squared_error: 7.6324 - val_loss: 1.5186 - val_mean_squared_error: 5.1630 - 50ms/epoch - 2ms/step
Epoch 17/50
24/24 - 0s - loss: 2.1391 - mean_squared_error: 7.5736 - val_loss: 1.5236 - val_mean_squared_error: 4.6890 - 50ms/epoch - 2ms/step
Epoch 18/50
24/24 - 0s - loss: 2.0327 - mean_squared_error: 6.3583 - val_loss: 1.4318 - val_mean_squared_error: 4.5316 - 50ms/epoch - 2ms/step
Epoch 19/50
24/24 - 0s - loss: 2.1677 - mean_squared_error: 7.6499 - val_loss: 1.4694 - val_mean_squared_error: 4.8320 - 58ms/epoch - 2ms/step
Epoch 20/50
24/24 - 0s - loss: 2.1430 - mean_squared_error: 7.5095 - val_loss: 2.0898 - val_mean_squared_error: 6.7728 - 58ms/epoch - 2ms/step
Epoch 21/50
24/24 - 0s - loss: 2.2006 - mean_squared_error: 7.4860 - val_loss: 2.4348 - val_mean_squared_error: 8.5576 - 50ms/epoch - 2ms/step

Epoch 23/50
24/24 - 0s - loss: 1.9980 - mean_squared_error: 6.8224 - val_loss: 1.5940 - val_mean_squared_error: 5.0737 - 70ms/epoch - 3ms/step
Epoch 24/50
24/24 - 0s - loss: 1.9468 - mean_squared_error: 6.2974 - val_loss: 1.3463 - val_mean_squared_error: 4.1348 - 42ms/epoch - 2ms/step
Epoch 25/50
24/24 - 0s - loss: 1.8930 - mean_squared_error: 5.8309 - val_loss: 1.3309 - val_mean_squared_error: 4.0374 - 50ms/epoch - 2ms/step
Epoch 26/50
24/24 - 0s - loss: 1.9790 - mean_squared_error: 6.3781 - val_loss: 1.3901 - val_mean_squared_error: 4.3671 - 50ms/epoch - 2ms/step
Epoch 27/50
24/24 - 0s - loss: 1.9150 - mean_squared_error: 5.8791 - val_loss: 1.2976 - val_mean_squared_error: 3.7185 - 50ms/epoch - 2ms/step
Epoch 28/50
24/24 - 0s - loss: 1.9545 - mean_squared_error: 6.1785 - val_loss: 1.3288 - val_mean_squared_error: 3.7235 - 50ms/epoch - 2ms/step
Epoch 29/50
24/24 - 0s - loss: 1.8869 - mean_squared_error: 6.0787 - val_loss: 1.9059 - val_mean_squared_error: 5.8556 - 50ms/epoch - 2ms/step

In [5]:
# sociodemographic predictors

y_dnn_socio_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

avg_r_squared = 0

def standarize_data(data, stats):
    return (data - stats['mean'])/ stats['std']

for train_index, test_index in ten_fold.split(X_sociodemo_test_passed):
    print("TEST:", test_index)

    X_train, X_test = X_sociodemo_test_passed.iloc[train_index], X_sociodemo_test_passed.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #scaler = StandardScaler()
    
    #scaled_X_train = scaler.fit_transform(X_train)
    #scaled_X_test = scaler.transform(X_test)
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    tf.random.set_seed(42)
    
    dnn_model = keras.models.Sequential([
        keras.layers.Dense(160,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(16,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(176,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128,activation="relu"),
        #keras.layers.Dropout(0.2),
        keras.layers.Dense(160,activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1)
    ])
    
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    dnn_model.compile(optimizer="adam", loss=keras.losses.mean_squared_error, metrics=[keras.metrics.mean_squared_error])
    dnn_model.fit(x=scaled_X_train, y=y_train, epochs=50, callbacks=[early_stop], verbose=2)
    
    this_y_predict = dnn_model.predict(scaled_X_test).flatten()

    y_dnn_socio_predict = y_dnn_socio_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [ 23  30  39  44  59  63  67  70  72  76  86  88  96 107 120 136 139 165
 168 198 208 209 215 218 247 250 259 260 265 275 280 292 294 298 310 312
 327 331 332 333 363 365 377 388 394 439 449 453 457 464 465 478 481 495
 500 513 518 519 527 541 554 559 569 589 591 599 616 617 643 644 653 658
 673 679 684 695 708 753 762 767 777 778 783 792 804 816 826 850 858 874
 883 908 926 932 942]
Epoch 1/50
27/27 - 1s - loss: 498.6574 - mean_squared_error: 498.6574 - 518ms/epoch - 19ms/step
Epoch 2/50
27/27 - 0s - loss: 76.2955 - mean_squared_error: 76.2955 - 33ms/epoch - 1ms/step
Epoch 3/50
27/27 - 0s - loss: 38.4647 - mean_squared_error: 38.4647 - 49ms/epoch - 2ms/step
Epoch 4/50
27/27 - 0s - loss: 30.6039 - mean_squared_error: 30.6039 - 34ms/epoch - 1ms/step
Epoch 5/50
27/27 - 0s - loss: 25.9365 - mean_squared_error: 25.9365 - 33ms/epoch - 1ms/step
Epoch 6/50
27/27 - 0s - loss: 18.9923 - mean_squared_error: 18.9923 - 44ms/epoch - 2ms/step
Epoch 7/50
27/27 - 0s - loss: 17.1195 - mean_square

Epoch 19/50
27/27 - 0s - loss: 8.0597 - mean_squared_error: 8.0597 - 47ms/epoch - 2ms/step
Epoch 20/50
27/27 - 0s - loss: 6.7704 - mean_squared_error: 6.7704 - 49ms/epoch - 2ms/step
Epoch 21/50
27/27 - 0s - loss: 6.9247 - mean_squared_error: 6.9247 - 43ms/epoch - 2ms/step
Epoch 22/50
27/27 - 0s - loss: 6.4831 - mean_squared_error: 6.4831 - 46ms/epoch - 2ms/step
Epoch 23/50
27/27 - 0s - loss: 6.8785 - mean_squared_error: 6.8785 - 44ms/epoch - 2ms/step
Epoch 24/50
27/27 - 0s - loss: 7.5867 - mean_squared_error: 7.5867 - 44ms/epoch - 2ms/step
Epoch 25/50
27/27 - 0s - loss: 6.0529 - mean_squared_error: 6.0529 - 48ms/epoch - 2ms/step
Epoch 26/50
27/27 - 0s - loss: 6.0648 - mean_squared_error: 6.0648 - 49ms/epoch - 2ms/step
Epoch 27/50
27/27 - 0s - loss: 5.9969 - mean_squared_error: 5.9969 - 49ms/epoch - 2ms/step
Epoch 28/50
27/27 - 0s - loss: 5.6883 - mean_squared_error: 5.6883 - 43ms/epoch - 2ms/step
Epoch 29/50
27/27 - 0s - loss: 6.2681 - mean_squared_error: 6.2681 - 49ms/epoch - 2ms/step

Epoch 8/50
27/27 - 0s - loss: 15.9915 - mean_squared_error: 15.9915 - 46ms/epoch - 2ms/step
Epoch 9/50
27/27 - 0s - loss: 13.3279 - mean_squared_error: 13.3279 - 43ms/epoch - 2ms/step
Epoch 10/50
27/27 - 0s - loss: 11.9159 - mean_squared_error: 11.9159 - 43ms/epoch - 2ms/step
Epoch 11/50
27/27 - 0s - loss: 11.0129 - mean_squared_error: 11.0129 - 45ms/epoch - 2ms/step
Epoch 12/50
27/27 - 0s - loss: 10.5839 - mean_squared_error: 10.5839 - 62ms/epoch - 2ms/step
Epoch 13/50
27/27 - 0s - loss: 9.6973 - mean_squared_error: 9.6973 - 52ms/epoch - 2ms/step
Epoch 14/50
27/27 - 0s - loss: 9.5699 - mean_squared_error: 9.5699 - 49ms/epoch - 2ms/step
Epoch 15/50
27/27 - 0s - loss: 8.7381 - mean_squared_error: 8.7381 - 44ms/epoch - 2ms/step
Epoch 16/50
27/27 - 0s - loss: 8.9573 - mean_squared_error: 8.9573 - 44ms/epoch - 2ms/step
Epoch 17/50
27/27 - 0s - loss: 7.7634 - mean_squared_error: 7.7634 - 44ms/epoch - 2ms/step
Epoch 18/50
27/27 - 0s - loss: 7.7757 - mean_squared_error: 7.7757 - 44ms/epoch - 

Epoch 6/50
27/27 - 0s - loss: 21.8848 - mean_squared_error: 21.8848 - 45ms/epoch - 2ms/step
Epoch 7/50
27/27 - 0s - loss: 18.2975 - mean_squared_error: 18.2975 - 43ms/epoch - 2ms/step
Epoch 8/50
27/27 - 0s - loss: 15.2613 - mean_squared_error: 15.2613 - 42ms/epoch - 2ms/step
Epoch 9/50
27/27 - 0s - loss: 13.9606 - mean_squared_error: 13.9606 - 50ms/epoch - 2ms/step
Epoch 10/50
27/27 - 0s - loss: 13.2247 - mean_squared_error: 13.2247 - 52ms/epoch - 2ms/step
Epoch 11/50
27/27 - 0s - loss: 11.6572 - mean_squared_error: 11.6572 - 49ms/epoch - 2ms/step
Epoch 12/50
27/27 - 0s - loss: 9.4654 - mean_squared_error: 9.4654 - 51ms/epoch - 2ms/step
Epoch 13/50
27/27 - 0s - loss: 9.4693 - mean_squared_error: 9.4693 - 49ms/epoch - 2ms/step
Epoch 14/50
27/27 - 0s - loss: 8.9834 - mean_squared_error: 8.9834 - 50ms/epoch - 2ms/step
Epoch 15/50
27/27 - 0s - loss: 9.5215 - mean_squared_error: 9.5215 - 50ms/epoch - 2ms/step
Epoch 16/50
27/27 - 0s - loss: 8.0297 - mean_squared_error: 8.0297 - 53ms/epoch - 

In [6]:
dnn_socio_rmse = mean_squared_error(y_true , y_dnn_socio_predict, squared=False)
dnn_socio_r2 = r2_score(y_true, y_dnn_socio_predict)
avg_r_squared = avg_r_squared/10

In [7]:
# sociodemographic - 10 epochs
print("RMSE of the DNN model with sociodemographic predictors: "+str(dnn_socio_rmse))
print("R2 of the DNN model with sociodemographic predictors: "+str(dnn_socio_r2)) # for R2, I use this one.
print("R2 of the DNN model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the DNN model with sociodemographic predictors: 1.495215483100996
R2 of the DNN model with sociodemographic predictors: 0.924333425255986
R2 of the DNN model with sociodemographic predictors (avg): 0.9239815852891397


In [8]:
dnn_socio_rmse = mean_squared_error(y_true , y_dnn_socio_predict, squared=False)
dnn_socio_r2 = r2_score(y_true, y_dnn_socio_predict)
avg_r_squared = avg_r_squared/10

In [9]:
# sociodemographic - 10 epochs
print("RMSE of the DNN model with sociodemographic predictors: "+str(dnn_socio_rmse))
print("R2 of the DNN model with sociodemographic predictors: "+str(dnn_socio_r2)) # for R2, I use this one.
print("R2 of the DNN model with sociodemographic predictors (avg): "+str(avg_r_squared))

RMSE of the DNN model with sociodemographic predictors: 1.495215483100996
R2 of the DNN model with sociodemographic predictors: 0.924333425255986
R2 of the DNN model with sociodemographic predictors (avg): 0.09239815852891396
